In [1]:
import pandas as pd
from rdframe.client.http_client import HttpClientDataFormat, HttpClient
from rdframe.knowledge_graph import KnowledgeGraph
from rdframe.dataset.rdfpredicate import RDFPredicate
from rdframe.dataset.aggregation_fn_data import AggregationData

In [2]:
    endpoint = 'http://10.161.202.101:8890/sparql/'
    port = 8890
    output_format = HttpClientDataFormat.PANDAS_DF
    max_rows = 1000000
    timeout = 12000
    default_graph_url = 'http://twitter.com'
    client = HttpClient(endpoint_url=endpoint,
                        port=port,
                        return_format=output_format,
                        timeout=timeout,
                        default_graph_uri=default_graph_url,
                        max_rows=max_rows
                        )

    graph = KnowledgeGraph('twitter',
                           'http://twitter.com/',
                           prefixes={
                               "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
                               "sioc": "http://rdfs.org/sioc/ns#",
                               "sioct": "http://rdfs.org/sioc/types#",
                               "to": "http://twitter.com/ontology/",
                               "dcterms": "http://purl.org/dc/terms/",
                               "xsd": "http://www.example.org/",
                               "foaf": "http://xmlns.com/foaf/0.1/"
                           })

    dataset = graph.entities(class_name='sioct:microblogPost',
                             new_dataset_name='tweets',
                             entities_col_name='tweet')
    ds = dataset.expand(src_col_name='tweet', predicate_list=[
        RDFPredicate('sioc:has_creater', 'tweep')
    ]).group_by(['tweep']).count(
        aggregation_fn_data=[AggregationData('tweet', 'tweets_count')]).filter(
        conditions_dict={'tweets_count': ['>= {}'.format(200), '<= {}'.format(300)]})

    ds = ds.expand(src_col_name='tweet', predicate_list=[
        RDFPredicate('sioc:content', 'text', optional=False),
        RDFPredicate('dcterms:created', 'date', optional=True),
        RDFPredicate('to:hasmedia', 'multimedia', optional=True),
        RDFPredicate('to:hashashtag', 'hashtag', optional=True),
        RDFPredicate('sioc:mentions', 'users_mentioned', optional=True)
    ])

    ds = ds.select_cols(['tweet', 'tweep', 'text', 'date', 'multimedia', 'hashtag', 'users_mentioned'])
    
    sparql_query = ds.to_sparql()    

In [3]:
    print("Sparql Query = \n{}".format(ds.to_sparql()))



Sparql Query = 
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX sioc: <http://rdfs.org/sioc/ns#>
PREFIX sioct: <http://rdfs.org/sioc/types#>
PREFIX to: <http://twitter.com/ontology/>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.example.org/>
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
SELECT ?tweet ?tweep ?text ?date ?multimedia ?hashtag ?users_mentioned 
FROM <http://twitter.com/>
WHERE 
{ 
	?tweet sioc:content ?text .
	?tweet rdf:type sioct:microblogPost .
	?tweet sioc:has_creater ?tweep .

	OPTIONAL {  
		?tweet dcterms:created ?date .
		?tweet to:hasmedia ?multimedia .
		?tweet to:hashashtag ?hashtag .
		?tweet sioc:mentions ?users_mentioned .
	}		
	{
		SELECT ?tweep 
		WHERE 
		{ 
			?tweet rdf:type sioct:microblogPost .
			?tweet sioc:has_creater ?tweep .
		
		} GROUP BY ?tweep
		 HAVING(( COUNT( ?tweet) >= 200 )  && ( COUNT( ?tweet) <= 300 ) ) 
	}
}


In [4]:
    df = ds.execute(client, return_format=output_format)
    print(df.head(10))

/Users/aisha/Desktop/QCRI/RDFrame/rdframe/dataset/dataset.py:192: DtypeWarning: Columns (3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  res = client.execute_query(query_string, return_format=return_format)


                                               tweet  \
0  http://twitter.com/5Yk7JzKkUUacrQv/status/1035...   
1  http://twitter.com/adeladelv/status/1035976828...   
2  http://twitter.com/alihks2999/status/103582758...   
3  http://twitter.com/sharida_a/status/1035796244...   
4  http://twitter.com/amzkh1/status/1036188540940...   
5  http://twitter.com/gdooh111/status/10327840499...   
6  http://twitter.com/justnow1991/status/10328100...   
7  http://twitter.com/justnow1991/status/10330713...   
8  http://twitter.com/yousef_tw/status/1032994882...   
9  http://twitter.com/yousef_tw/status/1032995374...   

                                 tweep  \
0  http://twitter.com/5Yk7JzKkUUacrQv/   
1        http://twitter.com/adeladelv/   
2       http://twitter.com/alihks2999/   
3        http://twitter.com/sharida_a/   
4           http://twitter.com/amzkh1/   
5         http://twitter.com/gdooh111/   
6      http://twitter.com/justnow1991/   
7      http://twitter.com/justnow1991/   
8    